In [1]:
import numpy as np
import pandas as pd
import glob
import ipyparallel as p
import os
import ntpath
import ProgressBar as pb
from pymail import alert
import time

In [2]:
exname = 'smex8_sen'
method = '8'
site = 'P301'

In [18]:
files = glob.glob('/Volumes/data/RHESSys_out/P301/%s/*_basin.daily'%exname)

In [19]:
len(files)

150000

In [20]:
params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

In [21]:
params.combo.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [22]:
params.head()

,MAP,PeakSWE,SMR,TpeakSWE,combo,idx,rd,ref,scenidx,sd
0,0.325,0.521211,0.014403,113.0,1,0,0.011104,2009,0,4.011104
1,0.325,0.521211,0.014403,113.0,2,0,0.111041,2009,1,4.111041
2,0.325,0.521211,0.014403,113.0,3,0,0.222082,2009,2,4.222082
3,0.325,0.521211,0.014403,113.0,4,0,0.444163,2009,3,4.444163
4,0.325,0.521211,0.014403,113.0,5,0,0.888326,2009,4,4.888326


In [23]:
files = pd.DataFrame({'basin_daily':files})

In [24]:
def getIDX(fl):
    return int(fl.split('_')[-2])

In [25]:
files['scenidx'] = files.basin_daily.map(getIDX)

In [26]:
params = pd.merge(params,files, on= 'scenidx')

In [27]:
from rhessys import snowmelt_experiment as smex

In [28]:
params['Tnosnow'] = params.TpeakSWE+np.ceil(params.PeakSWE/params.SMR)

In [29]:
params['date_peakSWE'] = params.apply(smex.DOPEAKSWE2cal,axis=1)
params['date_ONS'] =params.apply(smex.DONS2cal,axis=1)

In [30]:
c = p.Client()
view = c.load_balanced_view()

In [31]:
c.ids

[0, 1, 2, 3, 4, 5]

In [32]:
len(params)

150000

In [33]:
dr = pd.date_range('2008-10-1','2009-10-02',freq='D')

In [34]:
%px dr = pd.date_range('2008-10-1','2009-10-02',freq='D')

In [35]:
def processSM(bd,idx,strt,nd):

    # read in the basin daily data
    data = pd.read_csv(bd,delimiter=' ')

    data.index = pd.DatetimeIndex(dr)

    data['ET'] = data.evap + data.trans + data.snow_subl
    data['E'] = data.evap + data.snow_subl
    trans = data.trans.sum()
    #evap = data.evap.sum()
    evap = data.E.sum()
    ET = data.ET.sum() # compute total ET
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    # create full output object
    full = (idx,trans,evap,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    data = data.loc[strt:nd] # crop the data to the melt season
    T = data.trans.sum() # compute totoal transpiration
    #E = data.evap.sum() # compute total evaporation
    E = data.E.sum()
    ET = data.ET.sum() # compute total ET
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    melt = (idx,T,E,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    return full,melt

In [36]:
idx = 0
print params.basin_daily[idx]
processSM(params.basin_daily[idx],params.scenidx[idx],params.date_peakSWE[idx],params.date_ONS[idx])

/Volumes/data/RHESSys_out/P301/smex8_sen/smex8_sen_0_basin.daily


((0,
  34.84947,
  99.84390400000001,
  134.693374,
  616.4584530000001,
  230.15856499999998,
  386.299888,
  558.6806300000001,
  0.0,
  716.4708499999999,
  2.2134157356948063,
  2.732354108991829,
  1941.0120784932024,
  4.322964885558577,
  1946.6444373569475,
  4011.1040799999773,
  388.143045,
  11673.876054,
  0.2587964850136241,
  0.36701191825613094),
 (0,
  3.8754020000000002,
  10.646837000000001,
  14.522239000000003,
  68.89143899999999,
  31.740791000000005,
  37.15065,
  61.962256999999994,
  0.0,
  0.0,
  2.984500210526316,
  4.052827631578946,
  1942.07875531579,
  6.368756921052633,
  1946.6443399999994,
  4011.10408,
  70.650502,
  1366.295795,
  0.29263349999999994,
  0.3821641842105264))

In [37]:
#res = view.map(processSM,params.basin_daily[0:1000],params.scenidx[0:1000],params.date_peakSWE[0:1000],params.date_ONS[0:1000])
res = view.map(processSM,params.basin_daily,params.scenidx,params.date_peakSWE,params.date_ONS)

In [38]:
prog = pb.ProgressBar(len(params))

In [39]:
while res.ready() == False:
    prog.animate_ipython(res.progress)
    time.sleep(2)

#alert.send_alert('barnhatb@colorado.edu','Processing Nr1 %s has finished'%exname,'Your script has finished')

[****************100%******************]  149826 of 150000 complete                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         


In [40]:
res.done()

True

In [41]:
res.progress

150000

In [42]:
full,melt = zip(*res.result())

## Merge processing results with parameters dataset

In [43]:
idx,trans,evap,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*full)

params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,trans,evap,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['scenidx','trans','evap','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='scenidx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_full.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True)

In [35]:
params.head()

,MAP,PeakSWE,SMR,TpeakSWE,combo,idx,ksat,m,pa,po,...,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate
0,0.325,0.521211,0.014403,113.0,d1:s1,0,757.89,1.2548,0.050794,0.516732,...,3250.571043,0.018943,1099.917015,2.166730,4947.251389,11372.991699,397.726705,3.807343e+06,0.0,1.527166
1,0.325,0.521211,0.014403,113.0,d1:s2,0,757.89,1.2548,0.076191,0.516732,...,3101.757169,0.000000,1036.515251,46.963793,4880.200172,11218.850969,397.726705,3.807344e+06,0.0,1.527166
2,0.325,0.521211,0.014403,113.0,d1:s3,0,757.89,1.2548,0.101588,0.516732,...,2956.260288,0.000000,1001.767320,89.677185,4847.631834,11143.981228,397.726705,3.807344e+06,0.0,1.527166
3,0.325,0.521211,0.014403,113.0,d1:s4,0,757.89,1.2548,0.126985,0.516732,...,2803.562558,0.000000,976.767720,124.833347,4813.739275,11064.818086,397.726705,3.807344e+06,0.0,1.527166
4,0.325,0.521211,0.014403,113.0,d1:s5,0,757.89,1.2548,0.152382,0.516732,...,2658.737463,0.000000,905.667577,139.002862,4753.894760,10928.493701,397.726705,3.807344e+06,0.0,1.527166


In [44]:
len(params)

150000

In [45]:
idx,T,E,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*melt)

params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,T,E,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['scenidx','T','E','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='scenidx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_melt.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True) # 

In [38]:
params.shape

(250000, 31)

In [28]:
alert.send_alert('barnhatb@colorado.edu','Processing P301 %s has finished'%exname,'Your script has finished')